In [3]:
def some_func(*login_names):
    return {"login": l for l in login_names}

some_func("twitch_user1", "twitch_user2", "acb")

{'login': 'acb'}

In [ ]:
from multidict import MultiDict

def get_logins(*login_names):
    # Jeder login_name soll ein eigener 'login'-Key sein
    md = MultiDict()
    for name in login_names:
        md.add('login', name)
    return md

x = get_logins("twitch_user1", "twitch_user2", "acb")


'twitch_user1'